# Blueprint

## Preset Example

In [ ]:
import scipp as sc
%matplotlib widget
from beamlime.logging import get_scipp_logger

logger = get_scipp_logger(log_level="DEBUG", widget=True)
sc.display_logs()

In [ ]:
from beamlime.applications.daemons import DataProcessDaemon, ControlInterface
from beamlime.ready_factory import log_factory, app_factory
from beamlime.logging import BeamlimeLogger
from logging import DEBUG

In [ ]:
with app_factory.temporary_provider(BeamlimeLogger,lambda: logger):
    app_factory[BeamlimeLogger].setLevel(DEBUG)
    command = app_factory[ControlInterface]
    command.start()
    ctrl = app_factory[DataProcessDaemon]
    ctrl.run()

In [ ]:
from beamlime.applications.daemons import DataPlotter
app_factory[DataPlotter].show()

## Replace DataStreamListener and Workflow

In [ ]:
from beamlime.applications.providers import DataGenerator, Workflow, SimulationSetup
from beamlime.constructors import Factory
from dataclasses import dataclass
user_factory = Factory()

In [ ]:
@dataclass
class LocalSimulationSetup(SimulationSetup):
    num_data: int = 12
    max_pixels: int = 64
    random_seed: int = 123
    
user_factory.cache_product(SimulationSetup, LocalSimulationSetup)

In [ ]:
@user_factory.provider
def data_generator_provider(simulation_setup: SimulationSetup) -> DataGenerator:
    from numpy.random import default_rng

    st = simulation_setup
    rng = default_rng(st.random_seed)
    data_size = int(st.max_pixels)
    x_coord = sc.linspace("counts", start=0, stop=1, num=st.max_pixels, unit="m")
    y_coord = sc.linspace("counts", start=0, stop=1, num=st.max_pixels, unit="m")

    # Custom generator here
    def data_generator():
        for _ in range(st.num_data):
            values = rng.random(data_size)
            data = sc.array(dims=["counts"], values=values, unit="count")
            yield sc.DataArray(data=data, coords={"x": x_coord, "y": y_coord})

    return data_generator

In [ ]:
@user_factory.provider
def workflow_provider(simulation_setup: SimulationSetup) -> Workflow:
    from functools import reduce

    binsize = int(simulation_setup.max_pixels / 16)
    
    # Custom workflow here
    def workflow(data_chunk: list[sc.DataArray]):
        summed = reduce(lambda x, y: x + y, data_chunk)
        return summed.bin(x=binsize, y=binsize)

    return workflow

In [ ]:
with app_factory.local_factory(user_factory) as local_factory:
    with local_factory.temporary_provider(BeamlimeLogger,lambda: logger):
        local_factory[BeamlimeLogger].setLevel(DEBUG)
        command = local_factory[ControlInterface]
        command.start()
        ctrl = local_factory[DataProcessDaemon]
        ctrl.run()

In [ ]:
sc.display_logs()

In [ ]:
from beamlime.applications.daemons import DataPlotter
local_factory[DataPlotter].show()